In [ ]:
cd ..

In [2]:
%matplotlib inline

import cobra
import numpy as np
import escher
import optslope_rubisco
import pandas as pd
import pint
import seaborn as sns

from cobra.core import model
from cobra.flux_analysis import phenotype_phase_plane, pfba, flux_variability_analysis
from cobra.io import read_sbml_model
from cobra.io import sbml
from copy import deepcopy
from importlib_resources import read_text
from matplotlib import pyplot as plt
from typing import List, Tuple, Iterable, Any
from scipy import stats

## Flux Balance Analysis of CCMB1 + rubisco + prk metabolism
The purpose of this notebook is to 
* produce an FBA model of the CCMB1 strain with rubisco and prk;
* examine the contrbution of rubisco to 3PG production in this strain in silico.

We use parsimonious FBA (pFBA) to get a single defined flux solution for predictions.

In [3]:
# Read in the WT model - core e. coli model with rubisco and prk
WT_MODEL_FNAME = 'optslope_rubisco/core_model_with_rpp.xml'
wt_model = read_sbml_model(WT_MODEL_FNAME)

# Make a CCMB1 model: WT sans rpi, edd and eda activities
ccmb1_model = read_sbml_model(WT_MODEL_FNAME)
ccmb1_model.reactions.RPI.knock_out()
ccmb1_model.reactions.EDD.knock_out()

# Glycerol is converted to DHAP in E. coli, so we allow DHAP uptake
ccmb1_model.exchanges.EX_dhap_e.bounds = (-1000, 1000)

# Second model that disallows overflow metabolism entirely.
# This gives an upper limit of the fraction of 3PG production due to rubisco.
ccmb1_model_no_overflow = read_sbml_model(WT_MODEL_FNAME)
ccmb1_model_no_overflow.reactions.RPI.knock_out()
ccmb1_model_no_overflow.reactions.EDD.knock_out()
ccmb1_model_no_overflow.exchanges.EX_dhap_e.bounds = (-1000, 1000)

# Disallow overflow by disabling carbon exchange other than glycerol and CO2.
for ex in ccmb1_model_no_overflow.exchanges:
    # Leave glycerol and CO2 alone
    if ex.id in ('EX_dhap_e', 'EX_co2_e'):
        continue
        
    # Turn off all other carbon exchange
    mb = ex.check_mass_balance()
    if abs(mb.get('C', 0)) >= 1:
        ex.bounds = (0, 0)

In [4]:
# Diagram central metabolic fluxes for a single pFBA prediction for complemented CCMB1 grown on glycerol.
growth_obj = ccmb1_model.reactions.BIOMASS_Ecoli_core_w_GAM
s_max = pfba(wt_model, fraction_of_optimum=0.9999, objective=growth_obj)
escher.Builder(map_name="e_coli_core.Core metabolism", reaction_data=s_max.fluxes)

Builder(reaction_data={'ACALD': 0.0, 'ACALDt': 0.0, 'ACKr': 0.0, 'ACONTa': 0.0, 'ACONTb': 0.0, 'ACt2r': 0.0, '…

In [5]:
# Print exchange fluxes to make sure imports are reasonable.
# Notice that there is substantial secretion of acetate, formate, and ethanol in this example.
# Glycerol is predominantly metabolized aerobically by E. coli and selection for improved glycerol growth 
# tends to increase overflow metabolism (e.g. Cheng et al. Nat. Comm. 2014). For this reason we calculate
# flux predictions below with and without of overflow metabolism in order to get a plausible range. 
print('Growth rate', s_max.objective_value)
exs = wt_model.exchanges
for ex in exs:
    if abs(s_max.fluxes[ex.id]) > 0.1:
        print(ex, s_max.fluxes[ex.id])
        
# Summary of fluxes to/from 3PG helps figure out the directionality.
ccmb1_model.metabolites.get_by_id('3pg_c').summary(s_max.fluxes)

Growth rate 0.0


Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [6]:
# Diagram central metabolic fluxes for a single pFBA prediction for complemented CCMB1 grown on glycerol.
growth_obj = ccmb1_model.reactions.BIOMASS_Ecoli_core_w_GAM
s_max = pfba(ccmb1_model, fraction_of_optimum=0.9999, objective=growth_obj)
escher.Builder(map_name="e_coli_core.Core metabolism", reaction_data=s_max.fluxes)

Builder(reaction_data={'ACALD': -116.57174016330674, 'ACALDt': -1.3250000000011388, 'ACKr': -156.9210304676978…

In [7]:
# Print exchange fluxes to make sure imports are reasonable.
# Notice that there is substantial secretion of acetate, formate, and ethanol in this example.
# Glycerol is predominantly metabolized aerobically by E. coli and selection for improved glycerol growth 
# tends to increase overflow metabolism (e.g. Cheng et al. Nat. Comm. 2014). For this reason we calculate
# flux predictions below with and without of overflow metabolism in order to get a plausible range. 
print('Growth rate', s_max.objective_value)
exs = ccmb1_model.exchanges
for ex in exs:
    if abs(s_max.fluxes[ex.id]) > 0.01:
        print(ex, s_max.fluxes[ex.id])
        
# Summary of fluxes to/from 3PG helps figure out the directionality.
ccmb1_model.metabolites.get_by_id('3pg_c').summary(s_max.fluxes)

Growth rate 19873.31306018833
EX_ac_e: ac_e -->  156.92103046769782
EX_acald_e: acald_e -->  1.3250000000011388
EX_co2_e: co2_e <=>  664.4976547533506
EX_etoh_e: etoh_e -->  115.2467401633056
EX_h_e: h_e <=>  1000.0
EX_h2o_e: h2o_e <=>  -144.60130646373966
EX_nh4_e: nh4_e <=>  -229.16954162840167
EX_o2_e: o2_e <=>  -500.0
EX_pi_e: pi_e <=>  845.3920934587001
EX_dhap_e: dhap_e <=>  -1000.0


Percent,Flux,Reaction,Definition
80.27%,737.2,PGK,3pg_c + atp_c <=> 13dpg_c + adp_c
19.73%,181.3,RBC,co2_c + h2o_c + rubp__D_c --> 2.0 3pg_c + 3.0 h_c
Percent,Flux,Reaction,Definition
6.85%,-62.87,BIOMASS_Ecoli_core_w_GAM,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
93.15%,-855.6,PGM,2pg_c <=> 3pg_c


In [8]:
# Growth rate has default constraints because we don't have a measured uptake rate.
growth_obj = ccmb1_model.reactions.BIOMASS_Ecoli_core_w_GAM

# Run pFBA over a range "fraction_of_optimum" values to get a sense of viable fluxes.
# Here we use default media so overflow metabolism is allowed. We also vary the 
# ATP maintenance energy (the fixed cost of living) from zero to ≈25% of the total 
# flux to ATP which is roughly 3x the influx of glycerol. 
# NOTE: no reason to use f_opt < 0.8 as it doesn't affect the relative rubisco flux.
f_opt = np.arange(0.8, 0.991, 0.01).tolist() + np.arange(0.991, 1.001, 0.001).tolist()
maintenance_energies = np.arange(0, 751, 10)
opt_fluxes_overflow_allowed = []
f_opt_vals = []
atpm_lb_vals = []
overflow_allowed = []
for frac_of_opt in f_opt:
    for atpm_lb in maintenance_energies:
        ccmb1_model.reactions.ATPM.lower_bound = atpm_lb
        s_max = pfba(ccmb1_model, fraction_of_optimum=frac_of_opt, objective=growth_obj)
        
        opt_fluxes_overflow_allowed.append(s_max.fluxes)
        f_opt_vals.append(frac_of_opt)
        atpm_lb_vals.append(atpm_lb)
        overflow_allowed.append(True)
        
# Reset the bound on ATP maintenance to 0 for idempotence of the cell
ccmb1_model.reactions.ATPM.lower_bound = 0

In [9]:
# Make predictions without overflow metabolism.
# NOTE: when overflow is disabled, the rubisco flux fraction is independent of 
# the "fraction_of_optimum" parameter since the system has far fewer free variables,
# so we don't vary "fraction_of_optimum" here since there is no reason. 
growth_obj = ccmb1_model_no_overflow.reactions.BIOMASS_Ecoli_core_w_GAM
opt_fluxes_no_overflow = []

frac_of_opt = 0.99
for atpm_lb in maintenance_energies:
    ccmb1_model_no_overflow.reactions.ATPM.lower_bound = atpm_lb
    s_max = pfba(ccmb1_model_no_overflow, fraction_of_optimum=frac_of_opt, objective=growth_obj)
    
    opt_fluxes_no_overflow.append(s_max.fluxes)
    f_opt_vals.append(frac_of_opt)
    atpm_lb_vals.append(atpm_lb)
    overflow_allowed.append(False)
    
# Reset the bound on ATP maintenance to 0 for idempotence of the cell
ccmb1_model_no_overflow.reactions.ATPM.lower_bound = 0

In [10]:
# Reactions producing/consuming 3-phosphoglycerate in the ECC2+rubisco model
# rubisco (RBC) written in the 3pg producing direction - positive flux = production
# phosphoglycerate mutase (PGM) written in the 3pg producing direction - positive flux = production
# phosphoglycerate kinase (PGK) written in 3pg consuming direction - positive flux = consumption
fba_df = pd.DataFrame(opt_fluxes_overflow_allowed+opt_fluxes_no_overflow)
fba_df['fraction_of_optimum'] = f_opt_vals
fba_df['overflow_allowed'] = overflow_allowed
fba_df['atp_maintenance_lb'] = atpm_lb_vals

In [11]:
# Consistency checks and summary information
rbc_producing = fba_df.RBC >= 0
pgm_consuming = fba_df.PGM <= 0
pgk_producing = fba_df.PGK <= 0

# Check directional consistency
print('3pg producing reactions')
print('\tRubisco producing 3pg at all growth rates:', rbc_producing.all())
print('\tPGK producing 3pg at all growth rates:', pgk_producing.all())

print()
print('3pg consuming reactions')
print('\tPGM consuming 3pg at all growth rates:', pgm_consuming.all())

# Total 3pg production as a function of growth rate - produced by rubisco and pgk.
# Remember that PGK is written in the 3pg consuming direction, hence negative sign.
total_influx = (2*fba_df.RBC - fba_df.PGK)
flux_to_biomass = total_influx+fba_df.PGM

# total influx and outflux are equal of course. 
rub_pct = 100 * 2*fba_df.RBC / total_influx
pgm_pct = 100 * -fba_df.PGM / total_influx
pgk_pct = 100 * -fba_df.PGK / total_influx
# remaining flux to biomass goes to serine from 3pg
pct_to_ser = 100-pgm_pct

fba_df['rub_pct_3pg_prod'] = rub_pct
fba_df['pgk_pct_3pg_prod'] = pgm_pct
fba_df['pgm_pct_3pg_cons'] = pgm_pct
fba_df['ser_pct_3pg_cons'] = pct_to_ser

mean_rub_pct = rub_pct.mean()
mean_ser_pct = pct_to_ser.mean()
rub_pct_range = (fba_df.rub_pct_3pg_prod.min(), fba_df.rub_pct_3pg_prod.max())

print()
print('Mean percent of 3PG production flux through rubisco across all estimates: %.1f%%' % mean_rub_pct)
print('\tRange of rubisco percentages: (%.1f%%, %.1f%%)' % rub_pct_range)
print('Mean percent of 3PG consumption flux to serine across all estimates: %.1f%%' % mean_ser_pct)

# Save resulting dataframe
!mkdir -p notebooks/data/FBA
fba_df.to_csv('notebooks/data/FBA/ccmb1_fluxes.csv')
fba_df.head(5)

3pg producing reactions
	Rubisco producing 3pg at all growth rates: True
	PGK producing 3pg at all growth rates: True

3pg consuming reactions
	PGM consuming 3pg at all growth rates: True

Mean percent of 3PG production flux through rubisco across all estimates: 18.7%
	Range of rubisco percentages: (15.9%, 23.9%)
Mean percent of 3PG consumption flux to serine across all estimates: 6.5%


,ACALD,ACALDt,ACKr,ACONTa,ACONTb,ACt2r,ADK1,AKGDH,AKGt2r,ALCD2x,...,xu5p__D_t,2pg_t,dhap_t,fraction_of_optimum,overflow_allowed,atp_maintenance_lb,rub_pct_3pg_prod,pgk_pct_3pg_prod,pgm_pct_3pg_cons,ser_pct_3pg_cons
fluxes,0.0,0.0,-144.840752,142.396240,142.396240,-144.840752,0.0,106.117522,0.0,0.0,...,0.0,0.0,737.847663,0.8,True,0,21.560229,92.521308,92.521308,7.478692
fluxes,0.0,0.0,-146.251941,142.073834,142.073834,-146.251941,0.0,105.877256,0.0,0.0,...,0.0,0.0,737.916195,0.8,True,10,21.504502,92.540638,92.540638,7.459362
fluxes,0.0,0.0,-147.663130,141.751427,141.751427,-147.663130,0.0,105.636989,0.0,0.0,...,0.0,0.0,737.984727,0.8,True,20,21.448810,92.559956,92.559956,7.440044
fluxes,0.0,0.0,-149.074320,141.429021,141.429021,-149.074320,0.0,105.396723,0.0,0.0,...,0.0,0.0,738.053259,0.8,True,30,21.393154,92.579262,92.579262,7.420738
fluxes,0.0,0.0,-150.485509,141.106614,141.106614,-150.485509,0.0,105.156457,0.0,0.0,...,0.0,0.0,738.121791,0.8,True,40,21.337534,92.598555,92.598555,7.401445


In [12]:
# Print out the range of predictions with and without overflow for reporting in text.
cols = ['overflow_allowed', 'rub_pct_3pg_prod']
fba_df[cols].groupby('overflow_allowed').describe()

rub_pct_3pg_prod                                             \
                            count       mean       std        min        25%   
overflow_allowed                                                               
False                        76.0  21.844567  1.421668  18.902946  20.786895   
True                       2280.0  18.580418  1.374886  15.927093  17.499296   

                                                   
                        50%        75%        max  
overflow_allowed                                   
False             22.061345  23.047869  23.887157  
True              18.516577  19.536497  21.560229